This notebook is to generate `narration_mapping_fho2srl.csv` and `fho_narration_df_rows.json`. It could take hours to run. 

(You don't need this file if you have downloaded it as instructed in `from_clip_to_misalignsrl_samples.ipynb`)

In [ ]:
import json

file_path = "/home/yayuanli/fun/mistake_detection/fine_grained_action_mistake_detection/dataset/fho_main.json"

with open(file_path, "r") as file:
    fho_main_json = json.load(file)

# Now you can explore the structure of the loaded JSON data
# For example, you can print the keys of the top-level object
print(fho_main_json.keys())




dict_keys(['version', 'date', 'description', 'metadata', 'videos'])


In [ ]:
fho_main_json["videos"][1]["video_metadata"]

{'video_start_pts': 323,
 'video_base_numerator': 1,
 'video_base_denominator': 15360,
 'duration_sec': 3127.2,
 'num_frames': 93816,
 'fps': 30.0,
 'width': 1440,
 'height': 1080,
 'rotation': None}

In [ ]:
fho_main_json["videos"][1]["annotated_intervals"][0]["narrated_actions"][4]

{'warnings': [],
 'uid': 'ab7f112d-4e9f-4f5e-9326-ccf19f216d99',
 'start_sec': 33.721028600000004,
 'end_sec': 41.721028600000004,
 'start_frame': 1011,
 'end_frame': 1251,
 'is_valid_action': True,
 'is_partial': False,
 'clip_start_sec': 33.721028600000004,
 'clip_end_sec': 41.721028600000004,
 'clip_start_frame': 1011,
 'clip_end_frame': 1251,
 'narration_timestamp_sec': 37.7126996,
 'clip_narration_timestamp_sec': 37.7126996,
 'narration_text': '#C C places the steel bowl on a table',
 'narration_annotation_uid': 'e5866a1d-dbad-4da9-8776-0cb23082d744',
 'structured_verb': 'put_(place,_leave,_drop)',
 'freeform_verb': None,
 'state_transition': 'other_-_[chemical_change_(e.g.,_burn_paper),_temperature_change_(e.g.,_boiling_water)]',
 'critical_frames': {'pre_45': 1082,
  'pre_30': 1097,
  'pre_15': 1112,
  'post_frame': 1147,
  'contact_frame': None,
  'pre_frame': 1127,
  'pnr_frame': 1130},
 'clip_critical_frames': {'pre_45': 1082,
  'pre_30': 1097,
  'pre_15': 1112,
  'post_frame

In [ ]:
import pandas as pd
narration_df = "/z/home/yayuanli/dat/Ego4D_Mistake/v1/egoclip_narrations_exploed_groupby_no,txt.csv"
narration_df = pd.read_csv(narration_df, index_col=0)

In [ ]:
from tqdm import tqdm
# data["videos"][1]["annotated_intervals"][0]["narrated_actions"][0]["narration_text"]
fho_narration_df_rows = []

for v_i, video_dict in tqdm(enumerate(data["videos"]), total=len(data["videos"]), desc="making `fho_narration_df_rows`. progress by video"):
    video_uid = video_dict["video_uid"]
    
    for interval_i, big_clip_dict in enumerate(video_dict["annotated_intervals"]):
        
        for action_i, narration_clip_dict in enumerate(big_clip_dict["narrated_actions"]):

            narration_text = narration_clip_dict["narration_text"]
            narration_timestamp_sec = narration_clip_dict["narration_timestamp_sec"]
            fho_narration_df_rows.append({"video_uid": video_uid, 
                                          "narration_text": narration_text, 
                                          "narration_timestamp_sec": narration_timestamp_sec,
                                          "video_index": v_i,
                                          "interval_index": interval_i,
                                          "action_index": action_i})
            # print(f"{video_uid=}, {narration_text=}, {narration_timestamp_sec=}")
            
    #         if len(fho_narration_df_rows) > 10:
    #             break
    #     if len(fho_narration_df_rows) > 10:
    #         break
    # if len(fho_narration_df_rows) > 10:
    #     break
import pprint
# pprint.pprint(fho_narration_df_rows)

import numpy as np
# tolerance_sec = 1e-5
narration_mapping_fho2srl = {"video_uid": [], "narration_text": [], "narration_timestamp_sec": [], "srl_index": [], "fho_index": []}
# find the narration row in my narration_df
# making `narration_mapping_fho2srl_df`. progress by fho narr row: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 155369/155369 [5:02:57<00:00,  8.55it/s]
for fho_index, fho_narr_row in tqdm(enumerate(fho_narration_df_rows), total=len(fho_narration_df_rows), desc="making `narration_mapping_fho2srl_df`. progress by fho narr row"):
    video_uid = fho_narr_row["video_uid"]
    narration_text = fho_narr_row["narration_text"]
    narration_timestamp_sec = fho_narr_row["narration_timestamp_sec"]
    
    match_video_uid = narration_df["video_uid"] == video_uid
    match_narration_timestamp_sec = np.isclose(narration_df["narration_time"], narration_timestamp_sec)
    # match_narration_text = narration_df["clip_text"] == narration_text    
    
    # check if srl_narr table has corresponding row
    srl_index = np.where(match_video_uid & match_narration_timestamp_sec)[0] # a list of indices. could be 2 or more since the srl df refered here is the exploed version
    if len(srl_index) > 0:
        srl_index = srl_index
        narration_mapping_fho2srl["video_uid"].append(video_uid)
        narration_mapping_fho2srl["narration_text"].append(narration_text)
        narration_mapping_fho2srl["narration_timestamp_sec"].append(narration_timestamp_sec)
        narration_mapping_fho2srl["srl_index"].append(srl_index)
        narration_mapping_fho2srl["fho_index"].append(fho_index)
        
        # print(f"{video_uid=}, {narration_text=}, {narration_timestamp_sec=}, {srl_index=}")
    # print(fho_narr_row["srl_index"])
    # print(narration_df[match_video_uid & match_narration_timestamp_sec])
narration_mapping_fho2srl_df = pd.DataFrame(narration_mapping_fho2srl) # columns: video_uid(str), narration_text (srt), narration_timestamp_sec (float), srl_index (list)
narration_mapping_fho2srl_df



making `fho_narration_df_rows`. progress by video:   0%|          | 1/1725 [00:00<00:00, 11066.77it/s]
making `narration_mapping_fho2srl_df`. progress by fho narr row: 100%|██████████| 11/11 [00:01<00:00,  6.26it/s]


,video_uid,narration_text,narration_timestamp_sec,srl_index,fho_index
0,26202090-684d-4be8-b3cc-de04da827e91,#C C raises both hands,10.44777,[389561],0
1,26202090-684d-4be8-b3cc-de04da827e91,#C C opens a paper bag,34.66642,"[389562, 389563]",2
2,26202090-684d-4be8-b3cc-de04da827e91,#C C takes a steel bowl out of the paper bag,35.43174,"[389564, 389565]",3
3,26202090-684d-4be8-b3cc-de04da827e91,#C C places the steel bowl on a table,37.71270,"[389566, 389567]",4
4,26202090-684d-4be8-b3cc-de04da827e91,#C C dips his hand inside the paper bag,39.95216,[389568],6
5,26202090-684d-4be8-b3cc-de04da827e91,#C C takes his hand out of the paper bag,40.83746,[389569],7
6,26202090-684d-4be8-b3cc-de04da827e91,#C C carries a plastic container off a sack,43.54789,"[389570, 389571]",9
7,26202090-684d-4be8-b3cc-de04da827e91,#C C takes off the lid of the plastic container,44.28887,"[389572, 389573]",10


In [ ]:
# save `fho_narration_df_rows` (a list of dict) and `narration_mapping_fho2srl_df` (a pandas dataframe)
import json
import pandas as pd
# save `fho_narration_df_rows` (a list of dict) and `narration_mapping_fho2srl_df` (a pandas dataframe)
# Save fho_narration_df_rows as JSON
with open('fho_narration_df_rows.json', 'w') as f:
    json.dump(fho_narration_df_rows, f, indent=4)

# Save narration_mapping_fho2srl_df as CSV
narration_mapping_fho2srl_df.to_csv('narration_mapping_fho2srl_df.csv')

In [ ]:
# %%
# count how much narrations in fho could be found in srl df
print(f"{len(narration_mapping_fho2srl_df)} out of {len(fho_narration_df_rows)} fho narrations could be found in srl df. Ratio: {len(narration_mapping_fho2srl_df)/len(fho_narration_df_rows)}")
# 37945 out of 155369 fho narrations could be found in srl df. Ratio: 0.24422503845683502

# 3/4 of narrations in fho_main.json are mismatched. it'd be good to generate group_df and etc. directly from fho_main.json

In [ ]:
# # %%
# # read `fho_narration_df_rows` and `narration_mapping_fho2srl_df`
# print("Reading `fho_narration_df_rows` and `narration_mapping_fho2srl_df`...")
# narration_mapping_fho2srl_df = pd.read_csv('narration_mapping_fho2srl_df.csv', index_col=0)
# # narration_mapping_fho2srl_df["srl_index"] = narration_mapping_fho2srl_df["srl_index"].apply(lambda x: ast.literal_eval(x.strip().replace("[ ", "[").replace(" ", ",")))
# narration_mapping_fho2srl_df["srl_index"] = narration_mapping_fho2srl_df["srl_index"].apply(ast.literal_eval)
# # for string like "[443 895]" or "[ 54 92]", make them list-like so that they can be processed by ast.literal_eval()

# # read fho_narration_df_rows from fho_narration_df_rows.json
# with open('fho_narration_df_rows.json', 'r') as f:
#     fho_narration_df_rows = json.load(f)
    
# # %%
# # count how much narrations in fho could be found in srl df
# print(f"{len(narration_mapping_fho2srl_df)} out of {len(fho_narration_df_rows)} fho narrations could be found in srl df. Ratio: {len(narration_mapping_fho2srl_df)/len(fho_narration_df_rows)}")
# # 37945 out of 155369 fho narrations could be found in srl df. Ratio: 0.24422503845683502

#### Test

In [ ]:
import random

map_index = random.randint(0, len(narration_mapping_fho2srl_df)-1) # random.randint is inclusive
srl_index = np.random.choice(narration_mapping_fho2srl_df.loc[map_index, 'srl_index'])
display(narration_df.iloc[srl_index])
display(narration_mapping_fho2srl_df.iloc[map_index])

video_uid                 26202090-684d-4be8-b3cc-de04da827e91
video_dur                                          3127.233333
narration_source                              narration_pass_1
narration_ind                                                7
narration_time                                         40.8375
clip_start                                           40.583986
clip_end                                             41.090933
clip_text                C takes his hand out of the paper bag
tag_verb                                                  [93]
tag_noun                                        [321, 12, 349]
ARG0                                                         C
V                                                        takes
ARG1                                                  his hand
valid_tag_noun                                           [321]
valid_txt_noun                                        ['hand']
valid_tag_verb                                         

video_uid                      26202090-684d-4be8-b3cc-de04da827e91
narration_text             #C C takes his hand out of the paper bag
narration_timestamp_sec                                    40.83746
srl_index                                                  [389569]
fho_index                                                         7
Name: 5, dtype: object

In [ ]:
# utils.py
import ast
from tqdm import tqdm
import pandas as pd

def convert_column_from_str_to_list(df, column_name_list):
    """
    Convert a column of a DataFrame to a list.
    
    Args:
    df: DataFrame
    column_name: str
    
    Returns:
    list
    """
    # do all rows for each column: 2m
    for column_name in column_name_list:
        print(f"Converting column `{column_name}` to list...")
        if isinstance(df[column_name][0], str):
            tqdm.pandas(desc=f"`{column_name=}`")
            df[column_name] = df[column_name].progress_apply(ast.literal_eval)
    
    # do all columns for each rows: 6m
    # def xx(row, column_name_list):
    #     for column_name in column_name_list:
    #         row[column_name] = ast.literal_eval(row[column_name])
    #     return row
    
    # print(f"{column_name_list=}")
    # tqdm.pandas(desc=f"Converting columns to list...")
    # df = df.progress_apply(xx, args=(column_name_list,), axis=1) # axis=1: for each row
    
    return df

def convert_to_list(string):
    try:
        return ast.literal_eval(string)
    except:
        return []  # Return an empty list in case of error


def read_large_csv(file_path, columns_str2list=[], nrows=None, chunk_size=100, ):
    """
    Read a large CSV file in chunks.
    
    Args:
    """
    
    # Define the file path
    # file_path = f'../dataset/egoclip_groups_groupby_{grouping_type}.csv'

    # Determine the number of rows in the CSV
    total_rows = sum(1 for _ in open(file_path, 'r'))

    # Read the CSV with a progress bar
    # chunk_size = 100  # Adjust chunk size based on your needs
    chunks = []
    converters = None
    if columns_str2list != []:
        converters = {cname: convert_to_list for cname in columns_str2list}
    # Use tqdm to show progress
    for chunk in tqdm(pd.read_csv(file_path, 
                                chunksize=chunk_size,
                                index_col=0,
                                converters=converters,
                                nrows=nrows), 
                    total=total_rows/chunk_size):
        chunks.append(chunk)

    # Concatenate all chunks into a single DataFrame
    df = pd.concat(chunks, axis=0)

    # Now you can use df as a normal DataFrame
    return df    



In [ ]:
group_df = read_large_csv(f'/z/home/yayuanli/dat/Ego4D_Mistake/v1/egoclip_groups_groupby_no,txt.csv', 
                        #   columns_str2list=["narration_index", "narration_indices", "mismatch_noun", "mismatch_verb", "mismatch_verb_noun"], 
                        nrows=None)
group_df

100%|█████████▉| 449/449.88 [01:50<00:00,  7.46it/s]/home/yayuanli/fun/mistake_detection/TRAVEl/.venv/lib/python3.10/site-packages/tqdm/std.py:636: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
100%|██████████| 450/449.88 [01:50<00:00,  4.07it/s]


,txt_verb,txt_noun,narration_index,narration_indices,mismatch_noun,mismatch_verb,mismatch_verb_noun
0,add,apple,[1802844],[1802844],"[40, 42, 1702, 1704, 1708, 1711, 2494, 5253, 1...","[9660, 9700, 9702, 9714, 9715, 9716, 9789, 979...","[577091, 2343352, 1660733, 144395, 340720, 132..."
1,add,art,"[2643102, 2643106, 2643109, 2643116, 2643118]","[2643102, 2643106, 2643109, 2643116, 2643118]","[40, 42, 1702, 1704, 1708, 1711, 2494, 5253, 1...","[21342, 21343, 33628, 33630, 33631, 33632, 336...","[2006220, 866548, 1856044, 1874576, 1259197, 2..."
2,add,bag,[164070],[164070],"[40, 42, 1702, 1704, 1708, 1711, 2494, 5253, 1...","[1, 5, 10, 14, 16, 19, 36, 37, 43, 44, 1445, 1...","[2086713, 2410394, 2068189, 2274354, 1999364, ..."
3,add,ball,"[81153, 81188, 1025763]","[81153, 81188, 1025763]","[40, 42, 1702, 1704, 1708, 1711, 2494, 5253, 1...","[21247, 21249, 21250, 21251, 21252, 21254, 212...","[2304639, 1370921, 1509872, 1503577, 1676916, ..."
4,add,bamboo,[815019],[815019],"[40, 42, 1702, 1704, 1708, 1711, 2494, 5253, 1...","[22016, 22019, 22020, 22023, 22024, 22027, 220...","[2417270, 1400304, 669665, 2092866, 874299, 19..."
...,...,...,...,...,...,...,...
44982,zip,sweater,[892566],[892566],"[4751, 4752, 32204, 75030, 87290, 114329, 1147...","[14356, 14470, 14471, 14472, 14473, 14474, 144...","[2390286, 751299, 1064723, 2168898, 129729, 17..."
44983,zip,table,"[326974, 2322775]","[326974, 2322775]","[4751, 4752, 32204, 75030, 87290, 114329, 1147...","[1014, 1100, 1125, 1222, 1237, 1295, 1313, 131...","[1738742, 131197, 1444922, 657220, 2258942, 12..."
44984,zip,thigh,"[115507, 115510, 115531, 1912755]","[115507, 115510, 115531, 1912755]","[4751, 4752, 32204, 75030, 87290, 114329, 1147...","[40701, 65967, 87422, 87423, 87424, 87426, 114...","[1698814, 185773, 1423716, 2288823, 595399, 20..."
44985,zip,wallet,"[681486, 1340941]","[681486, 1340941]","[4751, 4752, 32204, 75030, 87290, 114329, 1147...","[46980, 74915, 74926, 75071, 75072, 75078, 750...","[1775813, 1937050, 445543, 2688820, 2583193, 1..."


In [ ]:
match_txt_verb = group_df["txt_verb"] == narration_df.iloc[srl_index]["valid_txt_verb_single"]
match_txt_noun = group_df["txt_noun"] == narration_df.iloc[srl_index]["valid_txt_noun_single"]
sampled_groups = group_df[match_txt_verb & match_txt_noun]
sampled_groups

,txt_verb,txt_noun,narration_index,narration_indices,mismatch_noun,mismatch_verb,mismatch_verb_noun
38552,take,hand,"[7277, 7292, 7818, 7819, 22159, 22161, 22671, ...","[7277, 7292, 7818, 7819, 22159, 22161, 22671, ...","[24, 163, 194, 195, 222, 226, 233, 243, 258, 2...","[55, 57, 59, 63, 64, 67, 69, 107, 108, 137, 14...","[2712979, 135580, 1513758, 2008197, 2630203, 1..."


In [ ]:
# given an index (srl_index) in narration_df, find the index 
type_name_col_name_map = {"MisalignSRL_V": "mismatch_verb", "MisalignSRL_ARG1": "mismatch_noun", "MisalignSRL_V_ARG1": "mismatch_verb_noun"}
mistake_example_meta_dict = {_: None for _ in type_name_col_name_map}

misalignsrl_type = "MisalignSRL_V"
for misalignsrl_type in type_name_col_name_map:
    index_list = sampled_groups[type_name_col_name_map[misalignsrl_type]].iloc[0] # list of int. the srl index in the srl narration df
    if not isinstance(index_list, list):
        index_list = ast.literal_eval(index_list)
    for srl_index in index_list:
        # find fho_index (for fho_narration_df_rows)
        # narration_mapping_fho2srl_df["srl_index"] is a column where each row is a list of int. srl_index is a int. Find the row number where the srl_index is in the list of the row
        fho_index_row = narration_mapping_fho2srl_df[narration_mapping_fho2srl_df["srl_index"].apply(lambda x: srl_index in x)] # a list of indices.
        if len(fho_index_row) > 0:
            fho_index = fho_index_row["fho_index"].iloc[0]
            mistake_example_meta_dict[misalignsrl_type] = fho_narration_df_rows[fho_index]
            break

pprint.pprint(mistake_example_meta_dict)

{'MisalignSRL_ARG1': {'action_index': 3,
                      'interval_index': 0,
                      'narration_text': '#C C takes a steel bowl out of the '
                                        'paper bag',
                      'narration_timestamp_sec': 35.4317396,
                      'video_index': 1,
                      'video_uid': '26202090-684d-4be8-b3cc-de04da827e91'},
 'MisalignSRL_V': {'action_index': 0,
                   'interval_index': 0,
                   'narration_text': '#C C raises both hands',
                   'narration_timestamp_sec': 10.447769599999999,
                   'video_index': 1,
                   'video_uid': '26202090-684d-4be8-b3cc-de04da827e91'},
 'MisalignSRL_V_ARG1': None}


In [ ]:
 fho_main_json["videos"][1]["annotated_intervals"][0]["narrated_actions"][0]

{'warnings': [],
 'uid': '289c1114-f2cf-483c-be2f-a7bbc30ddc5f',
 'start_sec': 6.454361933333334,
 'end_sec': 14.454361933333333,
 'start_frame': 193,
 'end_frame': 433,
 'is_valid_action': False,
 'is_partial': False,
 'clip_start_sec': 6.454361933333334,
 'clip_end_sec': 14.454361933333333,
 'clip_start_frame': 193,
 'clip_end_frame': 433,
 'narration_timestamp_sec': 10.447769599999999,
 'clip_narration_timestamp_sec': 10.447769599999999,
 'narration_text': '#C C raises both hands',
 'narration_annotation_uid': 'e5866a1d-dbad-4da9-8776-0cb23082d744',
 'structured_verb': None,
 'freeform_verb': None,
 'state_transition': None,
 'critical_frames': None,
 'clip_critical_frames': None,
 'frames': None,
 'is_rejected': True,
 'is_invalid_annotation': False,
 'reject_reason': '"no_human-object_interaction_from_camera_wearer_(e.g._c_looks_around,_c_pauses_work)"',
 'stage': 'critical_frame'}